# Linear Transformations

In [35]:
import random
import numpy as np

# Code Challenges

**1. Develop a python function from scratch that will find the determinants of any $n$ x $n$ matrix.**

In [36]:
def mat_det(A):
    assert len(A) == 0 or all(type(row) == list for row in A) and len(A) == len(A[0]), "A should be a square matrix."
    
    n = len(A)
    if   n == 0: return 1.
    elif n == 1: return A[0][0]
    elif n == 2: return A[0][0] * A[1][1] - A[0][1] * A[1][0] 

    det = 0.
    for col, cofactor in enumerate(A[0]):
        minor = [[A[i][j] for j in range(n) if j != col] for i in range(1, n)]
        det += (-1) ** col * cofactor * mat_det(minor)

    return det

**2. Develop a python function from scratch that will find both the eigenvectors and eigenvalues of any $n$ x $n$ matrix.**

In [37]:
def mat_eig(A, max_iterations = 50000):
    assert len(A) == 0 or all(type(row) == list for row in A) and len(A) == len(A[0]), "A should be a square matrix."

    n = len(A) 
    if n == 0: return [], []
    if n == 1: return [A[0][0]], [[1.]]

    eigenvectors = []
    eigenvalues = []

    for _ in range(n):
        # Generate random guess for eigenvector with ||eigvec|| = 1
        eigvec = [random.random() for _ in range(n)]
        norm_eigvec = sum(a ** 2 for a in eigvec) ** 0.5
        eigvec = [a / norm_eigvec for a in eigvec]

        # Power Method
        for _ in range(max_iterations):
            tf_eigvec = [sum(a * b for a, b in zip(row, eigvec)) for row in A]
            
            norm_tf = sum(a ** 2 for a in tf_eigvec) ** 0.5
            eigvec = [a / norm_tf for a in tf_eigvec]
            
        eigval = sum(a * b for a, b in zip(tf_eigvec, eigvec))
        eigenvalues.append(eigval)
        eigenvectors.append(eigvec)

        # Deflate Matrix with Weilandt deflation
        eigenval_outer_product = [[a * b for b in eigvec] for a in tf_eigvec]
        A = [[A[i][j] - eigenval_outer_product[i][j] for j in range(n)] for i in range(n)]

    return eigenvalues, [list(row) for row in zip(*eigenvectors)]

**3. Test your functions from a randomly generated $n$ x $n$ matrix.**

- Comparison my determinant to the `numpy`'s for an $n$ x $n$ matrix.<br>
Test 1:

In [38]:
A = np.random.random((8, 8)).astype("float64") * 5

my_result = mat_det(A.tolist())
np_result = np.linalg.det(A)

print("-- A --", A, sep="\n")
print("\n-- My det(A) --", my_result, sep="\n")
print("\n-- Numpy det(A) --", np_result, sep="\n")
print("\nAre my results equal to numpy?", "Yes." if np.isclose(my_result, np_result) else "No.")

-- A --
[[3.59579087 0.52234343 0.53467009 0.79377282 2.10952058 1.35392133
  4.97209715 1.32656723]
 [4.47441808 3.08122318 2.9305966  4.73423241 0.54526223 3.43746025
  4.02472296 2.95162468]
 [4.01087728 4.30179912 4.74100644 0.43211081 1.94955347 2.14610044
  3.17660395 1.78620921]
 [3.0052765  1.5652269  2.59061569 0.64487643 1.3736346  0.39744323
  3.5550859  1.90460094]
 [4.48382914 2.44710269 0.69001868 2.75007835 3.65191605 2.74242608
  3.47666059 4.45094663]
 [2.90598375 1.83505528 0.41216087 2.14346273 4.71984652 1.11880704
  4.25145167 1.32834167]
 [3.32868728 1.96962332 2.22905122 4.34694857 1.7803002  3.89229666
  3.72748889 1.40952619]
 [4.67181398 1.10693994 4.34093421 2.0215506  0.8909471  2.6789444
  1.49469963 1.44452728]]

-- My det(A) --
6415.088508069304

-- Numpy det(A) --
6415.088508069305

Are my results equal to numpy? Yes.


- Comparison of my eigenvalue and eigenvector algo to the `numpy`'s algo for an $n$ x $n$ matrix.
<br>
Test 2:

In [216]:
A = np.random.random((3, 3)).astype("float64") * 5

np_ev, np_evl = np.linalg.eig(A)
m_ev, m_evl = mat_eig(A.tolist())
m_ev, m_evl = np.array(m_ev), np.array(m_evl)

print("-- A --", A, sep="\n")
print("\n-- My Eigenvalues and Eigenvectors --", m_ev, m_evl, sep="\n")
print("\n-- Numpy Eigenvalues and Eigenvectors --", np_ev, np_evl, sep="\n")

print("\n\n", "Are the differences of eigenvalues close?", "Yes." if np.allclose(m_ev, np_ev) else "No.")

cosine_similarity = np.array([npe.dot(me) / (np.linalg.norm(me) * np.linalg.norm(npe)) for npe, me in zip(np_evl.T, m_evl.T)]).T
print("\n-- Cosine Similarity of Eigenvectors --", cosine_similarity, sep="\n")

-- A --
[[4.15989526 0.50810703 1.0431367 ]
 [4.08027029 1.64667948 0.72658524]
 [1.88339571 2.18610126 4.67200778]]

-- My Eigenvalues and Eigenvectors --
[6.79091678 2.14419339 1.54347236]
[[ 0.40323599 -0.4462923   0.79917862]
 [ 0.43365206 -0.87972187  0.59945293]
 [ 0.80582047  0.16405062 -0.04438156]]

-- Numpy Eigenvalues and Eigenvectors --
[6.79091678 2.14419339 1.54347236]
[[-0.40323599  0.20908245  0.08782629]
 [-0.43365206  0.65719782  0.79094366]
 [-0.80582047 -0.7241378  -0.60555319]]


 Are the differences of eigenvalues close? Yes.

-- Cosine Similarity of Eigenvectors --
[-1.         -0.79025844  0.57119778]
